In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


# Description

# Modules and Global Variables

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    TrainingArguments, Trainer,
)

import torch
import wandb

import datasets
import evaluate

import numpy as np
import pandas as pd

import os
import re
import random

from module.args import *

print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 1


In [3]:
### labels

labels = pc_binary_labels

label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

print(label2id)
print(id2label)

{'True': 0, 'False': 1}
{0: 'True', 1: 'False'}


In [4]:
### paths and names

PROJECT_NAME = 'aspect_sentiment_classification_binary'
RUN_ID = 'uncleaned_v21_maxlen_256'

DATA_V = 'uncleaned_v21'
DATA_T = 'pc_binary' # ce or pc or pc_binary
AUGMENTATION = False
AUG_NAME = ''

MAX_LEN = False

model_checkpoint = 'monologg/koelectra-base-v3-discriminator'

notebook_name = 'train_asc.ipynb'

### fixed

model_name = re.sub(r'[/-]', r'_', model_checkpoint).lower()
run_name = f'asc_{model_name}_{RUN_ID}'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', RUN_ID)
NOTEBOOK_PATH = os.path.join(ROOT_PATH, notebook_name)

augornot = f'_{AUG_NAME}' if AUGMENTATION is True else ''
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_train{augornot}.csv')
EVAL_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_dev.csv')

!mkdir -p {SAVE_PATH}

In [5]:
print_paths(PROJECT_NAME, model_checkpoint, DATA_V, SAVE_PATH, NOTEBOOK_PATH, TRAIN_DATA_PATH, EVAL_DATA_PATH)
print()
print_args()

PROJECT_NAME: aspect_sentiment_classification_binary

model_checkpoint: monologg/koelectra-base-v3-discriminator
DATA_V: uncleaned_v21

SAVE_PATH: ./training_results/uncleaned_v21_maxlen_256 exists.
NOTEBOOK_PATH: ./train_asc.ipynb exists.

TRAIN_DATA_PATH: ./dataset/uncleaned_v21/pc_binary_train.csv exists.
EVAL_DATA_PATH: ./dataset/uncleaned_v21/pc_binary_dev.csv exists.

per_device_train_batch_size: 32
per_device_eval_batch_size: 32
learning_rate: 0.0002
num_train_epochs: 10


# WandB Configuration

In [6]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=aspect_sentiment_classification_binary
env: WANDB_NOTEBOOK_NAME=./train_asc.ipynb
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Load Model, Tokenizer, and Collator

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)

if MAX_LEN:
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=MAX_LEN)
else:
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form]).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys())) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

print(len(new_tokens))
print(new_tokens)

35000





3060
35212


212
{'앝', '😲', '˂', '⁉', 'ɪ', '##🤘', '##잍', '💯', 'ㅖ', 'ᴇ', '😯', '횽', '🎂', '##ɢ', '##😬', '##💯', '##💇', '챦', '챳', 'ᴘ', 'ᴠ', '⏰', '🙃', '##ᴀ', '뜌', '##🖒', '💋', '🧚', '##💝', '##🕷', '##💞', '##🥤', '##😲', '💞', '펏', '##ᴠ', '🕸', '🍼', '😴', '##ᴡ', '똭', '👦', '🐄', '##쫜', '##͈', '옄', '쓩', '😶', '##🧚', 'ෆ', '##옄', '##츌', '쵝', '##쨕', 'ғ', 'ᴍ', '💎', '쨕', '##챦', '##˚', '🎵', '##ˇ', '##ᵕ', '촥', '##˃', '🍎', '곘', '➕', 'ʀ', '🤮', '➰', '🐱', 'ɢ', '##🎵', '̵', 'ᴗ', '##🤡', '👠', '💄', '☝', '뗴', '˚', '##ᴘ', '##👦', '##♬', '👌', '쫜', '컄', 'ღ', '##😶', '😮', '죱', 'ᵕ', '##👨', '##🕸', '##ɪ', '💬', '##젔', '##닠', '##죱', '💆', '##ꈍ', 'ᴜ', '🌹', '◍', '🤘', '##🐥', '##듕', '◡', '##앝', '##ᴏ', '🥤', '##ᴇ', '##🌝', '💧', '##뜌', '귯', '##◡', 'ˇ', '읒', '##̵', '🖐', '##ɴ', '##🌹', '##➰', '🤡', '##띡', '##㉦', 'ʜ', '닠', '##♩', '💝', '##ᴍ', '##®', '##닼', '㉦', '뿤', '##🚗', '쏨', '츌', '🎀', '##♪', '😺', '👨', '밪', '##쵝', '😬', '🍷', '핡', '##👆', '🌝', '띡', '😡', '🐥', '🕷', '잍', '##ᴜ', 'ᴛ', '🕺', '닼', '##🎂', '♩', '🖒', '##읒', '##펏', '##귯', '##◍', '##💆', '졓', '##💄', 'ᴏ', 'ᴀ

In [9]:
model.config.label2id, model.config.id2label, model.num_labels

({'True': 0, 'False': 1}, {0: 'True', 1: 'False'}, 2)

# Define Metrics

In [10]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [11]:
### multi
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
    
#     accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
#     f1_positive, f1_negative, f1_neutral = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1,2])['f1'])
#     f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
#     f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
#     return {'accuracy': accuracy, 
#             'f1_positive': f1_positive, 'f1_negative': f1_negative, 'f1_neutral': f1_neutral, 
#             'f1_macro': f1_macro, 'f1_micro': f1_micro}

### binary
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
    f1_true, f1_false = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1])['f1'])
    f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
    f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
    return {'accuracy': accuracy, 'f1_true': f1_true, 'f1_false': f1_false, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

# Load Data

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["form"], examples["pair"], truncation=True)

In [13]:
train_dataset = pd.read_csv(TRAIN_DATA_PATH)
eval_dataset = pd.read_csv(EVAL_DATA_PATH)
train_dataset = datasets.Dataset.from_pandas(train_dataset).shuffle(seed=42)
eval_dataset = datasets.Dataset.from_pandas(eval_dataset).shuffle(seed=42)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [14]:
len(train_dataset), len(eval_dataset)

(9588, 9006)

In [15]:
k = random.randrange(len(train_dataset))
print(tokenizer.decode(train_dataset['input_ids'][k]), train_dataset['labels'][k])
k = random.randrange(len(eval_dataset))
print(tokenizer.decode(eval_dataset['input_ids'][k]), eval_dataset['labels'][k])

[CLS] 다양한 고농축 약초와 허브추출물 함유로 수분공급, 보습은 물론 모근강화, 비듬 / 각질케어까지!! [SEP] 본품 # 품질 # neutral [SEP] 1
[CLS] 자연주의 수분보습제 ` 자일리틸글루코사이드 ` 를 함유하고 있어 속보습 유지하는데 최고👍 [SEP] 본품 # 품질 # negative [SEP] 1


# Load Trainer

In [16]:
args = TrainingArguments(
    output_dir=run_name,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)

In [17]:
# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # callbacks=[es],
)

# Run Trainer

In [19]:
trainer.train()
wandb.finish()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: pair, form, id. If pair, form, id are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.


***** Running training *****


  Num examples = 9588


  Num Epochs = 10


  Instantaneous batch size per device = 32


...

Configuration saved in /tmp/tmpjaknoqhl/config.json


Model weights saved in /tmp/tmpjaknoqhl/pytorch_model.bin


tokenizer config file saved in /tmp/tmpjaknoqhl/tokenizer_config.json


Special tokens file saved in /tmp/tmpjaknoqhl/special_tokens_map.json


eval/accuracy,██████▁███
eval/f1_false,██████▁███
eval/f1_macro,██████▁███
eval/f1_micro,██████▁███
eval/f1_true,██████▁███
eval/loss,█▅▁▇▁▃▆▁▃▃
eval/runtime,▇█▇▁▂▁▇█▇▁
eval/samples_per_second,▂▁▂█▇█▂▁▂█
eval/steps_per_second,▂▁▂█▇█▂▁▂█
train/epoch,▁▂▂▂▃▃▄▅▅▅▆▆▇▇▇███
train/global_step,▁▂▂▂▃▃▄▄▄▅▆▆▇▇▇███


In [20]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(run_name)
for ckpt in ckpts:
    ckpt = os.path.join(run_name, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

!mv {run_name} {SAVE_PATH}/